<a href="https://colab.research.google.com/github/githukelvin/converting-HF-models-to-GGUF/blob/main/Converting_HF_models_To_GGUF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook contains code and blocks to convert a hf models to GGUF

In [ ]:
# First clone llama.cpp git repository
!!git clone https://github.com/ggerganov/llama.cpp.git

In [ ]:
# Next step is to make the the repo and install requiremenst inrequirement.txt

!cd llama.cpp && LLAMA_CUBLAS=1 make && pip install -r requirements.txt

In [ ]:
# Login to huggingFace to allow you to download models and push the gguf to your repo
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# We will use Snapshot download from hugging face libray
from huggingface_hub import snapshot_download

In [ ]:
# Fisrt is the model you want to convert to gguf
# method type 8,4,16 bit quatinzation
# Location of where the model to be stored
# Location of where the quantized model to be stored
model_name="Jacaranda/UlizaLlama"
methods= [
   "q4_k_m"
]
base_model="./original_model"
quantized_path="./quantized_model"

In [ ]:
# Using snapshot download the model
snapshot_download(repo_id=model_name, revision="main", local_dir=base_model,local_dir_use_symlinks=False)

In [ ]:
original_model=quantized_path+"FP16.gguf"

In [ ]:
# Make directory to store the model
!mkdir ./quantized_model

In [ ]:
# The code line when executed is used to convert the model to gguf
!python llama.cpp/convert-hf-to-gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

In [ ]:
# This cell simply renames all file acording to methods passed

import os
for m in methods:
  qtype = f"{quantized_path}/{m.upper()}.gguf"
  os.system("./llama.cpp/quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
# This cell is for testing the gguf model
!./llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob-sw.txt

In [ ]:
# Now Uploading the gguf  to hugging face import necessary api
from huggingface_hub import HfApi,HfFolder,create_repo,upload_file

In [ ]:
model_path="./quantized_model/Q4_K_M.gguf"
repo_name="UlizaLlama_Q4_K_M-gguf"
repo_url=create_repo(repo_name,exist_ok=True,private=False)

In [ ]:
api= HfApi()

In [ ]:
# now pushing the model to huggingface
api.upload_file(
    path_or_fileobj=model_path,
    path_in_repo="Q4_K_M.gguf",
    repo_id="de-coder/UlizaLlama_Q4_K_M-gguf",
    repo_type="model"
    )